In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
print("DONE")

DONE


In [2]:
# data loading and pre processing
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


#importing the dataset
dataset=pd.read_csv('matches.csv')
print('DONE')

DONE


In [3]:
#getting the independent variables
X=dataset.iloc[:,:-1].values
#getting the dependent variable
Y=dataset.iloc[:,-1].values
print('DONE')
print(X.shape)

DONE
(2350, 52)


In [23]:
# Y=Y.astype(int)
# print(Y)
# print('DONE')

In [4]:
#train test split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)
print('DONE')

DONE


In [5]:
#scaling the inputs
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
X_train=ss.fit_transform(X_train)
X_test=ss.transform(X_test)
train_tensor_X = torch.from_numpy(X_train)
train_tensor_Y = torch.from_numpy(Y_train)

test_tensor_X = torch.from_numpy(X_test)
test_tensor_Y = torch.from_numpy(Y_test)
'''
#resizing the dataset to match torch constraints
train_tensor_X = train_tensor_X.view(-1,train_tensor_X.shape[0],train_tensor_X.shape[1])
test_tensor_X = test_tensor_X.view(-1,test_tensor_X.shape[0],test_tensor_X.shape[1])

train_tensor_Y = train_tensor_Y.view(-1,train_tensor_Y.shape[0])
'''


train_set_X = torch.utils.data.DataLoader(train_tensor_X, batch_size=5, shuffle = False)
test_set_X = torch.utils.data.DataLoader(test_tensor_X, batch_size=5, shuffle = False)

train_set_Y = torch.utils.data.DataLoader(train_tensor_Y, batch_size=5, shuffle = False)



print(train_tensor_Y.shape)
print('DONE')

# pre preocessing done
# got tensors for training and testing
# now we move on to define our NN structure


torch.Size([1880])
DONE


c:\users\nikhileshwar\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [6]:
# defining the structure of our neural network
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(52, 64)
        self.fc2 = nn.Linear(64,64)
        self.fc3 = nn.Linear(64,64)
        self.fc4 = nn.Linear(64,2)
    
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x),dim=1)
        return x
                
net = Net()
print(net)
        

Net(
  (fc1): Linear(in_features=52, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=2, bias=True)
)


In [7]:
import torch.optim as optim
optimizer = optim.Adam(net.parameters(),lr=0.001)
EPOCHS = 3

for epoch in range(EPOCHS):
    print('EPOCH--->', epoch)
    for (inp , out)in zip(train_set_X,train_set_Y):
        inp = inp.view(-1,inp.shape[1])
        #print(inp.shape)

        #out = out.view(-1,1)
        net.zero_grad()
        output = net(inp.float())
        loss = F.nll_loss(output,out.long())
        loss.backward()
        optimizer.step()
        #print(output)
   # print("LOSS--> " ,loss,inp.shape)
    print(output)
        

      

EPOCH---> 0
tensor([[8.6732e-04, 9.9913e-01],
        [1.0000e+00, 3.5772e-07],
        [1.6483e-03, 9.9835e-01],
        [9.9697e-01, 3.0331e-03],
        [6.3791e-01, 3.6209e-01]], grad_fn=<SoftmaxBackward>)
EPOCH---> 1
tensor([[7.1130e-05, 9.9993e-01],
        [9.9999e-01, 5.0715e-06],
        [3.6648e-05, 9.9996e-01],
        [9.7063e-01, 2.9368e-02],
        [9.5775e-03, 9.9042e-01]], grad_fn=<SoftmaxBackward>)
EPOCH---> 2
tensor([[5.8420e-06, 9.9999e-01],
        [1.0000e+00, 2.1740e-07],
        [4.6074e-05, 9.9995e-01],
        [9.9752e-01, 2.4811e-03],
        [4.6663e-04, 9.9953e-01]], grad_fn=<SoftmaxBackward>)


In [8]:
#train accuracy
import math
crct =0
for (row,orow) in zip(train_tensor_X,train_tensor_Y):
    inputt = row.view(-1,train_tensor_X[0].shape[0])
    outp = net(inputt.float())
    print("PREDICTED--> ",outp," ACTUAL--> ",orow)
    #print(outp[0][1])
    if(outp[0][0] > outp[0][1]):
        if(orow == 0):
            crct = crct+1
    elif(outp[0][1] > outp[0][0]):
        if(orow==1):
            crct = crct+1
print("ACCURACY : " ,crct / train_tensor_X.shape[0])        

PREDICTED-->  tensor([[1.0000e+00, 2.4470e-10]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[3.2135e-11, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 2.4718e-06]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.5724e-08, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.2094e-07, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 1.1159e-06]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[2.5542e-07, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 9.3608e-09]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[0.1302, 0.8698]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[5.7148e-04, 9.9943e-01]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.000

PREDICTED-->  tensor([[1.0000e+00, 6.4637e-09]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[4.1777e-06, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[0.7935, 0.2065]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[8.0138e-11, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.2407e-07, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 2.8648e-16]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[5.5092e-04, 9.9945e-01]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 1.4357e-07]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 1.0104e-06]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.8472e-07, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.000

PREDICTED-->  tensor([[8.9933e-09, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[7.8563e-15, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0555e-05, 9.9999e-01]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 1.6760e-06]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 2.2837e-12]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[9.9959e-01, 4.0683e-04]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[4.3769e-16, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 2.4066e-07]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[3.1372e-06, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[3.3115e-04, 9.9967e-01]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor

PREDICTED-->  tensor([[7.8962e-12, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[9.7898e-11, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 5.7313e-11]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.4520e-13, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 2.4122e-11]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[4.7333e-09, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[8.9711e-05, 9.9991e-01]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 9.4097e-11]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.3816e-10, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[8.4212e-05, 9.9992e-01]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor

PREDICTED-->  tensor([[0.0381, 0.9619]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[4.7809e-07, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[0.0010, 0.9990]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[0.9846, 0.0154]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.9558e-11, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[3.0217e-09, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[6.5748e-07, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.2696e-04, 9.9987e-01]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[3.5894e-07, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 1.5315e-07]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 1.6248e-0

...

PREDICTED-->  tensor([[4.3788e-11, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.7914e-05, 9.9998e-01]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 2.1683e-07]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.5728e-08, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[7.0955e-08, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[0.9975, 0.0025]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[9.9989e-01, 1.0983e-04]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[4.0801e-11, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[8.9693e-08, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.5306e-04, 9.9985e-01]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.000

PREDICTED-->  tensor([[1.0000e+00, 2.4808e-08]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 1.1706e-09]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[9.9992e-01, 7.5131e-05]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[0.0011, 0.9989]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[9.9980e-01, 2.0037e-04]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[9.9997e-01, 2.9884e-05]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 1.4227e-10]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[4.0593e-10, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[0.9988, 0.0012]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[5.7342e-14, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[2.2164e-04, 9

PREDICTED-->  tensor([[9.9999e-01, 5.8348e-06]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.6802e-07, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 1.1546e-07]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[9.9998e-01, 2.1147e-05]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.6718e-04, 9.9983e-01]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.3188e-06, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[9.9909e-01, 9.1272e-04]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 1.7085e-13]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[0.0577, 0.9423]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[7.7940e-07, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.000

PREDICTED-->  tensor([[1.6158e-04, 9.9984e-01]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 2.5156e-11]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[3.8960e-08, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[3.1372e-06, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 3.5504e-08]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 1.2198e-07]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 1.8670e-07]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.1579e-07, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[0.9981, 0.0019]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.2315e-07, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.000

PREDICTED-->  tensor([[1.0000e+00, 2.6362e-08]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 2.7630e-11]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[2.3414e-13, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[9.9993e-01, 6.7733e-05]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 9.6033e-10]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[0.0013, 0.9987]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[3.0280e-05, 9.9997e-01]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 1.1368e-18]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[7.0847e-08, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[5.9341e-06, 9.9999e-01]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.000

In [9]:
# test accuracy
import math
crct =0
for (row,orow) in zip(test_tensor_X,test_tensor_Y):
    inputt = row.view(-1,test_tensor_X[0].shape[0])
    outp = net(inputt.float())
    print("PREDICTED--> ",outp," ACTUAL--> ",orow)
    if(outp[0][0] > outp[0][1]):
        if(orow == 0):
            crct = crct+1
    elif(outp[0][1] > outp[0][0]):
        if(orow==1):
            crct = crct+1
print("ACCURACY : " ,crct / test_tensor_X.shape[0])        

PREDICTED-->  tensor([[4.2781e-10, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[3.7036e-10, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 4.1738e-08]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[8.1608e-15, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[9.9959e-01, 4.1067e-04]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[0.0043, 0.9957]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 2.9566e-09]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[9.2599e-05, 9.9991e-01]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 7.2164e-08]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[3.3236e-09, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[2.930

PREDICTED-->  tensor([[1.0000e+00, 1.5315e-07]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 8.3090e-11]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 1.3812e-12]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 1.5601e-11]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[0.9988, 0.0012]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[0.0011, 0.9989]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[0.9639, 0.0361]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 4.8996e-09]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 2.0297e-11]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 5.9026e-12]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 3.6227e-0

PREDICTED-->  tensor([[1.0000e+00, 6.1724e-08]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[0.0364, 0.9636]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.8012e-12, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 4.2246e-16]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 7.3973e-12]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[0.9903, 0.0097]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[7.5871e-14, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 9.5767e-11]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.2440e-05, 9.9999e-01]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[0.0681, 0.9319]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0767e-05, 9.9999e-0

PREDICTED-->  tensor([[1.1425e-09, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 1.8201e-14]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[4.2267e-07, 1.0000e+00]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[0.0371, 0.9629]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 4.8889e-10]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 2.4718e-06]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[1.0000e+00, 9.7004e-13]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[0.9971, 0.0029]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 4.5454e-16]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(0)
PREDICTED-->  tensor([[0.0014, 0.9986]], grad_fn=<SoftmaxBackward>)  ACTUAL-->  tensor(1)
PREDICTED-->  tensor([[1.0000e+00, 9.7191e-1

In [10]:
print(test_tensor_Y.shape,test_tensor_X.shape)


torch.Size([470]) torch.Size([470, 52])


In [11]:
torch.save(net, "./updated_weights.h5")
# abc = torch.load('./weights')

c:\users\nikhileshwar\appdata\local\programs\python\python36\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [71]:
# print(abc)
# #train accuracy
# import math
# crct =0
# for (row,orow) in zip(train_tensor_X,train_tensor_Y):
#     inputt = row.view(-1,train_tensor_X[0].shape[0])
#     outp = abc(inputt.float())
#     #print(outp[0][1])
#     if(outp[0][0] > outp[0][1]):
#         if(orow == 0):
#             crct = crct+1
#     elif(outp[0][1] > outp[0][0]):
#         if(orow==1):
#             crct = crct+1
# print("ACCURACY : " ,crct / train_tensor_X.shape[0]) 

Net(
  (fc1): Linear(in_features=52, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=2, bias=True)
)


In [16]:
print(net.state_dict())

OrderedDict([('fc1.weight', tensor([[-0.1514, -0.0949, -0.0133,  ..., -0.0940,  0.1145, -0.0446],
        [ 0.1036, -0.0701,  0.0443,  ..., -0.0721, -0.0011,  0.1565],
        [ 0.1922, -0.1298,  0.1246,  ..., -0.0290, -0.0490,  0.1618],
        ...,
        [-0.0637,  0.1192, -0.0539,  ..., -0.0842, -0.0223, -0.0747],
        [ 0.1255, -0.1349,  0.0115,  ..., -0.2259, -0.0040,  0.0935],
        [-0.0069, -0.0646, -0.0030,  ..., -0.0279,  0.0898, -0.0729]])), ('fc1.bias', tensor([-0.1297,  0.1002, -0.1280, -0.0009, -0.1202,  0.0329,  0.0390, -0.1392,
         0.1374,  0.0777,  0.1855,  0.1354, -0.0502, -0.1309, -0.1415,  0.1693,
        -0.0683, -0.0114,  0.0730,  0.0524,  0.0369,  0.0918, -0.0330,  0.0722,
        -0.0792,  0.0951, -0.1499,  0.0946,  0.1311, -0.0028,  0.0827,  0.0981,
         0.1607,  0.0508,  0.0619,  0.0153, -0.0067, -0.0150,  0.0670,  0.1157,
         0.0749,  0.0219, -0.0378,  0.0735, -0.0131, -0.0063, -0.0320, -0.0985,
         0.1665,  0.1043, -0.1837,  0.0489,